In [ ]:
Pos = tuple[int, int]

grid = dict()
with open("day23_input.txt") as file:
    for row, line in enumerate(file):
        for col, char in enumerate(line.strip()):
            grid[(row, col)] = char

START: Pos = (0, 1)
END: Pos = (row, col - 1)

In [ ]:
from typing import Generator


def get_neighbours_part1(
    pos: tuple[int, int]
) -> Generator[tuple[int, int], None, None]:
    row, col = pos
    match grid[pos]:
        case ">":
            d = [(0, 1)]
        case "<":
            d = [(0, -1)]
        case "^":
            d = [(-1, 0)]
        case "v":
            d = [(1, 0)]
        case _:
            d = [(0, 1), (0, -1), (-1, 0), (1, 0)]
    for dr, dc in d:
        nrow, ncol = row + dr, col + dc
        if ((nrow, ncol) not in grid) or (grid[(nrow, ncol)] == "#"):
            continue
        yield row + dr, col + dc

# Part 1


In [ ]:
def find_solutions(
    pos: tuple[int, int], end: tuple[int, int], visited: set, solutions: list[int]
) -> list[int]:
    if pos == end:
        solutions.append(len(visited))
    else:
        visited.add(pos)
        for npos in get_neighbours_part1(pos):
            if npos in visited:
                continue
            find_solutions(npos, end, visited, solutions)
        # Instead of giving each recursive call a copy of the visited-set, we just undo
        # the changes here. This saves a lot of memory and is faster.
        visited.remove(pos)

    return solutions

In [ ]:
print("Answer:", max(find_solutions(START, END, visited=set(), solutions=[])))

# Part 2


In [ ]:
def get_neighbours_part2(pos: tuple[int, int]) -> set[tuple[int, int]]:
    """We no longer care about the icy slopes. All neighbours are valid."""
    row, col = pos
    d = [(0, 1), (0, -1), (-1, 0), (1, 0)]
    neighbours = set()
    for dr, dc in d:
        nrow, ncol = row + dr, col + dc
        if ((nrow, ncol) not in grid) or (grid[(nrow, ncol)] == "#"):
            continue
        neighbours.add((nrow, ncol))
    return neighbours

Recursion is blowing up. On the paths in my input, you can walk for a long time before
reaching an intersection. Thus, to reduce the number of recursive calls, we can create a
graph of the maze and use DFS to test all possible ways to traverse the graph.


In [ ]:
from collections import defaultdict

# Queue: edge_start, current_pos, distance, visited
queue = [(START, START, 0, {START})]
graph = defaultdict(list[tuple[Pos, int]])

# Find all the junctions in the maze, and create a graph with edges between junctions.
while queue:
    edge_start, pos, distance, visited = queue.pop()
    neighbours = get_neighbours_part2(pos) - visited

    if len(neighbours) == 1:
        # We're on a path, not a junction.
        queue.append((edge_start, neighbours.pop(), distance + 1, visited | {pos}))
    else:
        # We have reached a junction or the end of the maze.
        if (pos, distance) in graph[edge_start]:
            # We have already found this edge, so we can skip it.
            continue

        # Add edge to the graph, in both directions.
        graph[edge_start].append((pos, distance))
        graph[pos].append((edge_start, distance))

        # Add all outgoing edges from the junction to the queue.
        for neighbour in neighbours:
            queue.append((pos, neighbour, 1, {pos}))

In [ ]:
def find_longest_path(pos, visited=set(), distance=0, longest_path=0):
    if pos == END:
        return max(longest_path, distance)
    else:
        visited.add(pos)
        for npos, ndistance in graph[pos]:
            if npos in visited:
                continue
            longest_path = find_longest_path(
                npos, visited, distance + ndistance, longest_path
            )
        visited.remove(pos)

    return longest_path

In [ ]:
print("Answer:", find_longest_path(START))